## 11.4 Transformer 架构

---
这是第一版没有的内容,好好看看,

---

2017 年开始一个新的架构开始在大多数 nlp 任务中超越 cnn.Transformer (deepl 老是翻译成变形金刚..)

Vaswani 等人的论文 `Attention is all you need` 第一次提到了 Transformer 架构,文章的要点就在标题中: 一个被称为 `neural attention` 的简单机制被证明可用用来构建强大的序列模型,这样模型不具备任何 cnn 或 rnn 的结构.

这一发现引发了 nlp 的一场革命,`neural attention` 的思想迅速成为深度学习最具影响力的思想之一.这一节我们会了解 `neural attention` 是如何工作的,了解它为何对序列数据如此有效.然后我们会创建一个 Transformer 编码器,这是 Transformer 架构的基础组件,随后将其应用在 imdb 电影评论分类任务.


## 理解 self-attention

阅读本书的过程中,你可能会选择略过部分,而认真阅读其他部分,这取决于你的目标或兴趣.如果这个过程能移植到你的模型呢?

模型看到的所有信息对手头任务的重要性并不相同,模型应该多注意某些信息,对其他特征更少关注.

听起来有些熟悉,我们已经在前文中两次接触到了相关的概念

- cnn 中的最大池化,只选择一个最大的特征代替原来整个部分.这是一种全有/全无的形式,只保留最重要特征,抛弃其他.
- tf-idf 归一化,根据不同标记可能携带信息的多少分配重要性标签,重要的标记会有更大的权重,不相关的标记则被淡化.

集中注意力的不同形式可能有很多,但都几乎都是从计算一组特征的重要性开始的.高的数值代表更相关的特征,低的数值代表更不相关的特征,这些数值该如何计算,以及我们该如何使用这些数值,依照不同的方法存在差异.

![general_attention](general_attention.png)

上图显示了深度学习中注意力机制: 输入特征被分配了注意力分数,用于告知输入的下一个表示.


最重要的是这样的关注机制不仅可以用来突出/抹去某些特征,还可以赋予特征上下文感知能力.

上一节中我们了解了词嵌入,词嵌入捕获不同的单词形成具有形状的向量空间,在这个嵌入空间中单词有固定的位置,与空间中其他单词形成固定关系集.但是这并不完全是自然语言的运行法制.自然语言中,一个词具体的含义通常非常依赖上下文的语境.

- 当提到 'date',这个 'date' 究竟是日期,还是约会?
- 提到 'see',究竟是 'I’ll see you soon' 见到,'I’ll see this the project to its end' 见证 'I see what you mean' 了解,直接将上面的 see 全部译为中文 '看见' 再对照译文也是一样的,`see` 和 '看见' 在这些语境下都有些微的差别.
- 又或者完全是指代的词 'it' 'she' 'in' 这些词的意思完全依赖上下文的语境.

显然,一个聪明的嵌入空间将为一个词的不同意思编码为不同的向量,这取决于它周围词的含义.这就是 self-attention 的作用,self-attention 目的是通过序列中相关标记来调节一个标记的表示.这就产生了上下文感知的标记特征.


考虑下面的例子: 'The train left the station on time.' 中的 'station' 究竟是什么含义? 火车站? 广播站? 甚至是国际空间站?这就是一个必须通过上下文语境才能明确含义的词.

![self_attention](self_attention.png)

第一步: 计算 'station' 向量与句子中其他每个词的相关度数字,这些是上文提到的注意力分数.我们简单的使用两个词向量直接的典籍衡量它们之间的关系,这是一个非常有效的计算距离的函数.早在 Transformers 架构出现之前,类似的方式已经是衡量两个词向量距离的标准方式了.当然在实践中,点积的数值还会经过一个缩放函数和 softmax,但是现在在这里,后续的处理都是一些细节了.

第二步: 计算句子中所有单词向量的总和,由相关性分数求加权总和.与 'station' 相关的词对最好结果影响最大(包括 'station' 本身),不相关的词几乎对结果没有什么影响.由此我们得到了一个新向量,这个向量由 'station' 和它的上下文语境而来,特别是它包含了 'train' 这个词向量的一部分,所以向量表示的是 火车站的含义.


如果对句子中每个词都重复这个过程,就产生了一个新的向量序列编码整个句子.下面是这个过程使用 numPy 类似形式的伪代码.

```py
def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape)
    for i, pivot_vector in enumerate(input_sequence): #遍历序列的每一个标记
        scores = np.zeros(shape=(len(input_sequence),))
        for j, vector in enumerate(input_sequence):
            scores[j] = np.dot(pivot_vector, vector.T) #计算这个标记向量与其他标记向量的距离(点积)
        scores /= np.sqrt(input_sequence.shape[1] )#归一化系数
        scores = softmax(scores)
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
            new_pivot_representation += vector * scores[j] #加权求和
        output[i] = new_pivot_representation # 最终结果
    return output
```


In [3]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

当然与其他情况相同,keras 早就由内置的实现了 -> MultiHeadAttention 层.示例代码如下:

In [ ]:
# num_heads = 4
# embed_dim = 256
# mha_layer = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
# outputs = mha_layer(inputs, inputs, inputs)

代码有一些奇怪...

- 为什么要反复传入 inputs 3次? 之前没见过这样的用法.
- 为什么这个层叫做 MultiHead?砍掉一个还能长出新的吗?(Hail HYDRA!)

这两个问题都有答案.


### 广义 self-attention: 查询-键-值 模型

到目前为止我们只考虑一个输入序列,然而 Transformer 开始是为了机器翻译开发的,需要两个输入序列:目前正在翻译的源序列("How’s the weather today?") 和你要将其转换为的目标序列("¿Qué tiempo hace hoy").

Transformer 是一个序列到序列的模型,这样的模型被设计用来将输入序列转换为另一个序列.(11.5 节会介绍到).


$$
outputs = sum({inputs_C} * pairwise_scores(inputs_A,inputs_B))
$$

现在让我们回到 self-attention 上,看看其内在执行方式.

3 个输入: input(A) input(B) input(C).

- 计算 输入A 与 输入B 的每个标记的相似程度.
- 使用这些分数对输入 C 的标记进行加权

非常重要的是没有任何强制 A B C 必须完全一致的要求,一般情况下使用 3 个不同的序列完成这个任务.分别称为 查询-键-值,这个操作的意义就可以解释为: 对每个查询中的元素,计算该元素与每个键值的关联程度,最后使用这些关联程度数字总和来衡量这个元素.

![query_key_value_2](query_key_value_2.png)


![retrieving_images_from_db](retrieving_images_from_db.png)

查询-键-值 这些术语来自搜索引擎和推荐系统.想象一些,现在你正在检索你收藏的所有图片,希望找到 '海滩上的狗',数据库内每张图片都有一组关键词描述--'猫' '狗' '聚会' 等等,我们把这些关键词称为键.搜索引擎会从比较查询和数据库的键,按照匹配度--相关性这些对键进行排序,并顺序返回前 N 个匹配项的对应图片.

从概念上说,这就是 类Transformer 注意力所关注做的事情.首先有一个知识体,我们试图从中提取信息(值),每个值都会有一个键,这个键始终都是以非常方便比较的格式存储,我们输入一个要寻找东西的参考序列(查询),然后数据库就能返回一组可能的结果.


实践中,键-值通常是同一序列,例如在机器翻译中,目标序列是查询,而源序列同时是 键 和 值: 对于目标的每个元素(例如: tiempo),这里要回到源序列("How’s the weather today?"),确认与目标距离最接近的元素("tiempo" 和 "weather" 应该是最相关的).

又或者,如果只是做序列分类任务,这样查询-键-值都会是同一序列: 把一个序列同自己比较,用整个序列的上下文充实每个标记.

这里解释了为什么我们要传递给 MultiHeadAttention 3次 input.那么 "multi-head" 又是指什么?


## Multi-Head attention

Multi-Head attention 是对 self-attention 的额外调整,首次引入是在 'Attention is all you need' 论文中.Multi-Head 意思在 self-attention 的输出空间分解为一组独立的子空间,分布执行 self-attention 查询-键-值 的步骤,最终的输出被串联成一个输出序列,每个这样的子空间被称为一个 Head.


![mha_layer](mha_layer.png)

中间有一段看不懂,无法翻译,略过.

这样原理上类似深度可分离卷积: 深度可分离卷积中,卷积的输出空间被分解为许多子空间(每个输出通道就是一个子空间),这些子空间被独立学习.

有一段看不懂,无法翻译,略过.


## Transformer 编码器

如何改进模型效果?

- 开始是增加额外的密集层
- 此时模型已经比较大了,增加残差链接,防止信息跨层传递丢失.
- 层的归一化可以帮助梯度在反向传播中更好的流动
- ...

以上可能就是 Transformer 发明之在构建 Transformer 的思维过程.大致如此: 将输入分为多个独立空间,增加残差连接,增加规范层,这些都是非常标准的架构模式,在任何模型中使用这些是非常明智的选择.

上面提高的这些内容共同构成了 Transformer 编码器,这是 Transformer 架构非常关键的两个部分之一.


Transformer 编码器是将 MultiHeadAttention 层和密集层连接,增加了规范化和残差连接.

![transformer_encoder](transformer_encoder.png)

原始的 Transformer 架构分类两个部分: Transformer 编码器(处理源序列) 和 Transformer 解码器.

最重要的是编码器非常通用,因此可以用于文本分类.可以接受一个序列并学习将其转换为更加有效的表示.接下来我们会使用 Transformer 编码器在 imdb 分类任务中试用.


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim  #输入的维度
        self.dense_dim = dense_dim  #密集层大小
        self.num_heads = num_heads  #head 的数量
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)  #MultiHeadAttention 层
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])  #密集层
        self.layernorm_1 = layers.LayerNormalization()  #归一化层
        self.layernorm_2 = layers.LayerNormalization()  #归一化层

    def call(self, inputs, mask=None):  #实现 call 方法
        if mask is not None:  # 小细节: 词嵌入层生成的 mask 是二维的,但是 attention 层的期望是 3/4 维,这里要扩大 mask 维度
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):  #序列化,可以通过这个方法保存模型
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

上面是原始版本 Transformer 解码器的自行实现.

注意这里归一化措施使用的是 LayerNormalization 层而不是在前文图片处理章节我们使用的 BatchNormalization 层.原因是 BatchNormalization 对序列数据归一化效果并不好.numpy 的伪代码如下

```py
def layer_normalization(batch_of_sequences):# 输入形状 (batch_size, sequence_length, embedding_dim)
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)# 我们只汇集最后一个轴上的数据.
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
    return (batch_of_sequences - mean) / variance
```

BatchNormalization 的 numpy 伪代码

```py
def batch_normalization(batch_of_images): # 输入形状 (batch_size, height, width, channels).
    mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))# 在批次轴 0 上汇集数据,批次中的样本之间产生了作用.
    variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
    return (batch_of_images - mean) / variance
```

BatchNormalization 会从许多样本中收集信息,获取特征均值和方差更准确的估计.而 LayerNormalization 会将每个序列独立于批次的其他序列进行规范化,单独汇集每个序列的数据,因此更适合序列数据的归一化.


In [2]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(
    x)  # TransformerEncoder 返回的是完整的序列,我们通过全局池化将每个序列减少维一个单例向量,方便分类.
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         5120000   
_________________________________________________________________
transformer_encoder (Transfo (None, None, 256)         543776    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 5,664,033
Trainable params: 5,664,033
Non-trainable params: 0
___________________________________________________

In [2]:
from tensorflow import keras

batch_size = 32  #批次大小

train_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)  #训练集
val_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)  #验证集
test_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)  #测试集

text_only_train_ds = train_ds.map(lambda x, y: x)  #只有原始文本信息的数据集

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [3]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_length = 600  #前600字
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,  #取词频前 20000的词
    output_mode="int",  #返回整数序列
    output_sequence_length=max_length,  #前600字以后截断输入
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))  #数据集
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

In [6]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=20,
          callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 62s 94ms/step - loss: 0.4811 - accuracy: 0.7786 - val_loss: 0.3536 - val_accuracy: 0.8482
Epoch 2/20
625/625 [==============================] - 54s 87ms/step - loss: 0.3063 - accuracy: 0.8711 - val_loss: 0.3393 - val_accuracy: 0.8626
Epoch 3/20
625/625 [==============================] - 55s 88ms/step - loss: 0.2347 - accuracy: 0.9039 - val_loss: 0.2862 - val_accuracy: 0.8900
Epoch 4/20
625/625 [==============================] - 55s 88ms/step - loss: 0.1833 - accuracy: 0.9280 - val_loss: 0.3170 - val_accuracy: 0.8902
Epoch 5/20
625/625 [==============================] - 56s 90ms/step - loss: 0.1503 - accuracy: 0.9434 - val_loss: 0.4423 - val_accuracy: 0.8704
Epoch 6/20
625/625 [==============================] - 55s 88ms/step - loss: 0.1271 - accuracy: 0.9518 - val_loss: 0.3689 - val_accuracy: 0.8874
Epoch 7/20
625/625 [==============================] - 52s 82ms/step - loss: 0.1116 - accuracy: 0.9586 - val_loss: 0.4802 - val_accuracy:

In [6]:
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 28s 34ms/step - loss: 0.3120 - accuracy: 0.8735
Test acc: 0.874


最终是 87.4% 和书上的 87.5% 相当接近.同样比 gru 要低.


上面的模型有一些不对劲的地方:

- 首先是准确率并没有提升,还不如 gru 模型.
- 使用的 Transformer 编码器.似乎并没有用到输入序列的顺序.
  - 密集层不关心顺序
  - MultiHeadAttention 层只是将输入看作是集合,也没有顺序.

Transformer 上文提到过是一个混合的序列模型,那处理序列那部分呢? 模型本身并不直接处理输入的顺序,我们只能人工注入语序信息.这部分叫做 '位置编码'(positional encoding)


![nlp_models_property_table](nlp_models_property_table.png)

上图总结了 nlp 模型的一些特点.

Transformer 是唯一即关心词序也关心上下文的模型.


### 使用位置编码重新注入语序信息

位置编码的背后思想很简单: 现在已经有了与上下文关联表示词的张量,我们只需要在平行的放入一个表示单词位置的向量就能将词序信息输入模型.至于位置向量如何编码这就是另一个问题了.

最简单的方案: 直接将词的位置填入位置向量,序列第一个词就是 0,第二个词就是 0 ..

这样做的问题: 神经网络不喜欢非常大的整数输入,而原始的位置信息可能会取到非常大的整数.

论文 'Attention is all you need paper' 使用了一个非常有趣的位置编码方案: 位置向量的值域是 `[-1,1]`,具体值根据位置不同而循环变化(余弦函数实现),这个编码方案非常聪明,使用一个小数值的项目来唯一描述大范围内的任何整数.

但是这并不我们在这个例子采用,我们使用的更简/有效的方案.人工设计编码方案费力,那让神经网络自己学习吧...

我们会嵌入一个位置向量


In [7]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim,
                 **kwargs):  #位置嵌入缺点是必须事先知道序列长度
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim,
                                                 output_dim=output_dim)  #嵌入层
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)  #标记位置嵌入层
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions  #两个嵌入层相加

    def compute_mask(
            self,
            inputs,
            mask=None):  #与直接使用 Embedding 一样,这个层应该能够生成 mask,这样就能忽略掉填充的数据,减少运算量.
        return tf.math.not_equal(inputs, 0)

    def get_config(self):  #保存模型时调用
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

我们定义了 PositionalEmbedding ,接下来会像使用普通嵌入层一样使用它.


#### 保存自定义层时注意事项

编写自定义图层时,务必确保实现 `get_config` 方法.这个方法在保存加载模型时会被调用到.

`get_config` 方法返回一个 python dict 其中包含了创建该层时构造函数需要的值.

所有的层都可以通过上面的方法进行序列化/反序列化.

```py
config = layer.get_config()
new_layer = layer.__class__.from_config(config)
```

PositionalEmbedding 的实例.

```py
layer = PositionalEmbedding(sequence_length, input_dim, output_dim)
config = layer.get_config()
new_layer = PositionalEmbedding.from_config(config)
```

保存包含自定义层的模型时,自定义层的配置会以字典形式保存在文件内.当从文件恢复模型时,我们需要向程序提供自定义层的类,确保程序能正确恢复自定义层.

```py
model = keras.models.load_model(
    filename, custom_objects={"PositionalEmbedding": PositionalEmbedding})
```


### 一个 Transformer 的文本分类器

这里我们将上文提高的所有东西组合在一起.组成一个考虑词序的 Transformer 模型,实际上就是将原来的 Embedding 替换成 PositionalEmbedding.


In [8]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
positional_embedding (Positi (None, None, 256)         5273600   
_________________________________________________________________
transformer_encoder (Transfo (None, None, 256)         543776    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 5,817,633
Trainable params: 5,817,633
Non-trainable params: 0
___________________________________________________

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=20,
          callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 58s 92ms/step - loss: 0.4648 - accuracy: 0.7888 - val_loss: 0.2971 - val_accuracy: 0.8756
Epoch 2/20
625/625 [==============================] - 58s 92ms/step - loss: 0.2288 - accuracy: 0.9132 - val_loss: 0.4022 - val_accuracy: 0.8552
Epoch 3/20
625/625 [==============================] - 58s 93ms/step - loss: 0.1751 - accuracy: 0.9349 - val_loss: 0.2788 - val_accuracy: 0.8912
Epoch 4/20
625/625 [==============================] - 60s 96ms/step - loss: 0.1436 - accuracy: 0.9475 - val_loss: 0.3324 - val_accuracy: 0.8888
Epoch 5/20
625/625 [==============================] - 62s 99ms/step - loss: 0.1216 - accuracy: 0.9564 - val_loss: 0.3441 - val_accuracy: 0.8856
Epoch 6/20
625/625 [==============================] - 58s 92ms/step - loss: 0.1047 - accuracy: 0.9628 - val_loss: 0.3865 - val_accuracy: 0.8744
Epoch 7/20
625/625 [==============================] - 55s 89ms/step - loss: 0.0939 - accuracy: 0.9671 - val_loss: 0.4254 - val_accuracy:

In [11]:
model = keras.models.load_model("full_transformer_encoder.keras",
                                custom_objects={
                                    "TransformerEncoder": TransformerEncoder,
                                    "PositionalEmbedding": PositionalEmbedding
                                })
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 27s 35ms/step - loss: 0.3204 - accuracy: 0.8737
Test acc: 0.874


最终结果是 87.4% 似乎没啥用啊,还降低了 0.1%...

书上的结果是 88.3% 比之前好了一些,这里没有复现出来.


### 序列模型和词袋模型之争

我们在 imdb 电影评论的文本分类任务,实际使用了序列模型和词袋模型.那么实际工程上这两种模型该如何取舍呢?

你或许听说过,词袋模型已经过时了,现在看到的很多深度学习 nlp 任务都是 Transformer 的序列模型一把梭子.但是通过本章的例子,我们能看出 Transformer 的序列模型也并非万能药.本章问题上表现最好的是词袋模型.

2017 年,作者和他的团队对很多不同类型的文本数据集进行分析,提出了一个文本分类问题的经验法则:

- 始终关注训练数据 样本数/样本的平均字数 的比例
- 比例 > 1500 使用序列模型
- 比例 < 1500 使用词袋模型

imdb 文本分类任务中样本数量 20000 平均字数 233,比例远远小于 1500 所以词袋模型更合适.

直觉上:

- 序列模型的输入代表了一个信息量更丰富更大的空间,同时需要更多的数据映射这个空间.同时词袋模型输入仅仅是无序的集合,其输入的空间非常简单,可能仅仅需要几百到几千个样本就能完成逻辑回归的训练.
- 样本越短,模型越不能丢弃样本包含的任何信息,特别是词序信息.'this movie is the bomb' 和 'this movie was a bomb' 仅有一个单词差别,使用词袋模型进行分类会非常困难,但是又词序信息的序列模型就很容易分辨.
- 另一方面,当样本越长,词的统计会变得可靠.这一点仅仅从词频统计直方图上就能看出来.

无论如何请谨记: 这个原则仅仅适用于文本分类,不一定适用于其他 nlp 任务,特别是涉及到机器翻译是,序列越长,相比其他架构 Transformer 的表现往往会越好.同时这个原则也仅仅是经验法则,未被严格的数学证明.这个原则并不一定在所有文本分类任务都成立.